In [34]:
!pip install skillNer
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 760.4 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [35]:
import spacy
import en_core_web_lg
from spacy.matcher import PhraseMatcher

# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor
from skillNer.general_params import SKILL_DB


# init params of skill extractor
nlp = en_core_web_lg.load()
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [36]:
# extract skills from job_description
job_description = """
 Rushabh Shah
Raleigh, NC  | rsshah7@ncsu.edu  | (919) 539 -7895 | www.linkedin.com/in/rushabh -shah -b66b66211
Skills

Languages  and Database : Python  ,HTML , CSS, JavaScript , C, MySQL
AI and Machine Learning : Scikit Learn, Tensorflow, NLTK, OpenCV
Certifications: Data Visualisation with Python ( IBM), Machine Learning with Python ( IBM), Python Bootcamp ( NPTEL ),
AWS Cloud Practitioner ( Amazon  Web Services ) , Interactivity with JavaScript( University of Michigan ), Generative AI
Fundamentals (Google  Cloud )
Work Experience

Machine Learning  Developer  Intern , Agilo Research Pvt. Ltd.  (STEMpedia) , India ,   Jan ’23 – May ‘23
• Played a key role in establishing and designing the AI Robotics Lab of STEMpedia, along with developing the Natural Language
Processing, Generalized Classification feature for the Pictoblox Educational Software.
• Developed and executed Python -based Machine Learning algorithms proficiently, prioritizing accessibility and
comprehensibility for students.
Cyber Security Management Intern , Alcumus ISO-QAR , India,      June ’22 - July ‘22
• Demonstrated proficiency in interpreting ISO -27001 and subsequently developing an analogous framework as per the
company requirement.
• Proposed  a comprehensive Quantitative Risk Assessment framework to identify , assess and mitigate security vulnerabilities
within business processes.
Front End Developer , Code Karo Yaaro , India        April ’21 - June ‘21
• Crafted and formulated webpages for an NGO's website in alignment with specified requisites in a team. The NGO's
requirement was centred around creating a website that would help advancing women's empowerment and providing
complimentary education to young children in rural areas.
• Utilized HTML, CSS, and JavaScript languages to establish the website  including different webpages for services and funding
options for the NGO . Collaborated with the developers to ensure the needs of the organisation are satisfied.
Projects

Education

Masters in Computer Science           May  ‘25
North Carolina State University , Raleigh
Coursewor k: Automated Learning And Data Analysis, Design Analysis and Algorithm, Cloud Architecture
B. Tech in Computer Engineering           May 2023
Pandit Deendayal Energy University , Gandhinagar, India        CGPA: 9.63
"""

annotations = skill_extractor.annotate(job_description)

In [37]:
skill_extractor.describe(annotations)

In [38]:
print(annotations)
data = annotations

{'text': 'rushabh shah raleigh nc | rsshah7@ncsu edu | 919 539 7895 | www linkedin com in rushabh shah b66b66211 skills languages and database python html css javascript c mysql ai and machine learning scikit learn tensorflow nltk opencv certifications data visualisation with python ibm machine learning with python ibm python bootcamp nptel aws cloud practitioner amazon web services interactivity with javascript university of michigan generative ai fundamentals google cloud work experience machine learning developer intern agilo research pvt ltd stempedia india jan ’23 – may ‘23 • played a key role in establishing and designing the ai robotics lab of stempedia along with developing the natural language processing generalized classification feature for the pictoblox educational software • developed and executed python based machine learning algorithms proficiently prioritizing accessibility and comprehensibility for students cyber security management intern alcumus iso qar india june ’2

In [39]:

# Extracting 'doc_node_value' field into a list
doc_node_values = [result['doc_node_value'] for result in data['results']['full_matches']]
doc_node_values += [result['doc_node_value'] for result in data['results']['ngram_scored']]

print(doc_node_values)


['machine learn', 'scikit learn', 'machine learn', 'google cloud', 'machine learn', 'natural language processing', 'educational software', 'machine learn', 'machine learn algorithm', 'cyber security', 'cyber security management', 'business process', 'front end', 'computer science', 'computer engineering', 'com', 'HTML', 'CSS', 'IBM', 'IBM', 'HTML', 'www', 'python', 'javascript', 'c', 'mysql', 'tensorflow', 'nltk', 'opencv', 'python', 'python', 'python', 'aws cloud practitioner', 'cloud amazon', 'amazon web', 'amazon web', 'web services', 'interactivity', 'javascript', 'research', 'robotics', 'executed', 'python', 'prioritizing', 'accessibility', 'risk framework', 'mitigate', 'vulnerabilities', 'requisites', 'empowerment', 'javascript', 'collaborated', 'analysis analysis', 'algorithm', 'cloud architecture', 'b']


In [40]:
import csv
csv_file = "doc_node_values.csv"
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['doc_node_value'])
    for value in doc_node_values:
        writer.writerow([value])

In [41]:
!pip install fuzzywuzzy

In [42]:
import pandas as pd
from fuzzywuzzy import fuzz

def load_skills_from_csv(file_path):
    # Load the skills from CSV
    return pd.read_csv(file_path)

def check_similarity(skill_resume, all_skills, threshold=100):
    # Check similarity using fuzzywuzzy library
    similarities = [fuzz.token_set_ratio(skill_resume, skill_data) for skill_data in all_skills]
    max_similarity = max(similarities)

    if max_similarity >= threshold:
        return True
    else:
        return False


# Load the skills from the resume and dataset CSV files
resume_skills = load_skills_from_csv('doc_node_values.csv')['doc_node_value']  # Replace with your actual file path
all_skills = load_skills_from_csv('lemmatized_skills.csv')['Lemmatized Skills']  # Replace with your actual file path
# Initialize a list to store matching skills
matching_skills = []

    # Check similarity for each skill in the resume
for skill_resume in resume_skills:
    if check_similarity(skill_resume, all_skills):
        matching_skills.append(skill_resume)

# Display the matching skills
#print("Matching Skills:", len(matching_skills), matching_skills
final_skills = set(matching_skills)
print("Matching skills : ",matching_skills)
print("length is : ",len(matching_skills))
print('\n')
print("Final skills : ",final_skills)
print("length is : ",len(final_skills))

Matching skills :  ['google cloud', 'natural language processing', 'business process', 'front end', 'HTML', 'HTML', 'python', 'javascript', 'c', 'mysql', 'tensorflow', 'python', 'python', 'python', 'javascript', 'research', 'robotics', 'python', 'javascript', 'analysis analysis', 'algorithm', 'cloud architecture', 'b']
length is :  23


Final skills :  {'front end', 'google cloud', 'robotics', 'cloud architecture', 'python', 'mysql', 'analysis analysis', 'algorithm', 'b', 'business process', 'c', 'javascript', 'natural language processing', 'HTML', 'tensorflow', 'research'}
length is :  16


In [43]:
from fuzzywuzzy import fuzz

# Function to compute percentage match between skills in resume and job description
def compute_percentage_match(resume_skills, job_description_skills):
    # Initialize counters
    matched_count = 0
    total_skills_in_resume = len(resume_skills)

    # Check similarity for each skill in resume with job description skills
    for skill_resume in resume_skills:
        for skill_job_desc in job_description_skills:
            similarity = fuzz.token_set_ratio(skill_resume, skill_job_desc)
            # If similarity score is above threshold, consider it as a match
            if similarity >= 60:  # Adjust threshold as needed
                matched_count += 1
                break  # Move to the next skill in resume

    # Compute percentage match
    percentage_match = (matched_count / total_skills_in_resume) * 100
    return percentage_match

# Extract skills from the job description
job_description_skills = ["Programming", 'Python', 'HTML', 'CSS', 'JavaScript', 'C','MySQL' , 'Scikit Learn', 'Tensorflow', 'NLTK', 'AWS Cloud',
'Front end Development',
'Cybersecurity Management']

# Compute percentage match between resume skills and job description skills
percentage_match = compute_percentage_match(final_skills, job_description_skills)

# Print the percentage match
print("Percentage match between resume skills and job description skills:", percentage_match)


Percentage match between resume skills and job description skills: 56.25


In [44]:
!pip install gensim
!pip install -U spacy
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [49]:

import spacy

# Load the large English model with word vectors
nlp = spacy.load('en_core_web_lg')

# Function to compute percentage match between skills in resume and job description using word embeddings
def compute_percentage_match(resume_skills, job_description_skills, nlp):
    # Initialize counters
    matched_count = 0
    total_skills_in_resume = len(resume_skills)

    # Check similarity for each skill in resume with job description skills
    for skill_resume in resume_skills:
        for skill_job_desc in job_description_skills:
            similarity = nlp(skill_resume).similarity(nlp(skill_job_desc))
            # If similarity score is above threshold, consider it as a match
            if similarity >= 0.55:  # Adjust threshold as needed
                matched_count += 1
                break  # Move to the next skill in resume

    # Compute percentage match
    percentage_match = (matched_count / total_skills_in_resume) * 100
    return percentage_match

# Extract skills from the job description
job_description_skills = [ 'python', 'HTML', 'CSS', 'JavaScript', 'C','MySQL' , 'Scikit Learn', 'tensorflow', 'NLP', 'Cloud',
'Front end Development',
'Cybersecurity Management']
# Extract skills from the resume (assuming matching_skills is already defined)

# Compute percentage match between resume skills and job description skills using word embeddings
percentage_match = compute_percentage_match(final_skills, job_description_skills, nlp)

# Print the percentage match
print("Percentage match between resume skills and job description skills:", percentage_match)


Percentage match between resume skills and job description skills: 68.75
